In [1]:
'''
*********************************************
this code produces cloud-mask binary data products for all MISR image blocks that we used in SIR prediction.
*********************************************

steps:
load the model and use the trained model for prediction.
produce cloud-mask binary files.
then we use these cloud-mask binary files in the SIR.c code and predict the SIR 
and check if SIR predictions have improved

inputs: red-band from 9 cameras

output: predicted cloud-mask for each block

to-do:
check file type of cloudmask inside SIR code; what format does input cloudmask file have in that code?
for-loop for the whole code
make a list of POB form all available toa-refls to keep count 

ok- extract POB fro toa-refl
ok- fillvalue from previous MISR cloudmasks?
ok- skip a pixel if there is [-1]?
ok- update the the predicted categorical list with fillvalue using indexing and assignment
***********************
? check 0,1 output in cloudmask output file; shoudl be binary, not float? 
>>> output cloudmask format is now [int8] [8 bits== range[-128,127]]; -2 is the fillvalue

? check format of this cloudmask as input file to the next code (atmmodel?) dtype= uint8? or int8?


then:
? run for a cloudy image and compare with baseline==visual conformation


finally- I need a code to read a cloudmask binary file and update the traning dataset for a specific cloudy block and then update the specific column for future comparison between my model and MISR cloudmasks


'''

'\n*********************************************\nthis code produces cloud-mask binary data products for all MISR image blocks that we used in SIR prediction.\n*********************************************\n\nsteps:\nload the model and use the trained model for prediction.\nproduce cloud-mask binary files.\nthen we use these cloud-mask binary files in the SIR.c code and predict the SIR \nand check if SIR predictions have improved\n\ninputs: red-band from 9 cameras\n\noutput: predicted cloud-mask for each block\n\nto-do:\ncheck file type of cloudmask inside SIR code; what format does input cloudmask file have in that code?\nfor-loop for the whole code\nmake a list of POB form all available toa-refls to keep count \n\nok- extract POB fro toa-refl\nok- fillvalue from previous MISR cloudmasks?\nok- skip a pixel if there is [-1]?\nok- update the the predicted categorical list with fillvalue using indexing and assignment\n***********************\n? check 0,1 output in cloudmask output file; 

In [2]:
import os
from glob import glob
import numpy as np
from keras.models import load_model
from matplotlib import pyplot as plt
import MisrToolkit as Mtk
from MisrToolkit import *

In [3]:
# load input files for the model
# inputs: 9 TOA reflectance red bands- order is important

toa_refl_dir = "/Users/ehsanmos/MLP_dataset/cloudmask_lab/prediction_test_cloudmask/test_cloudmask_p167_b10_july"

output_dir = toa_refl_dir

In [4]:
# set path to model & directory

model_name = "model_July_loss0.25_acc0.91.h5"
model_dir = "/Users/ehsanmos/MLP_dataset/cloudmask_lab/training_dataset"

In [5]:
# load the model for april/july

model_fp = os.path.join(model_dir, model_name) 
# load model
mlp_model_best = load_model(model_fp)
# summarize model.
mlp_model_best.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                640       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 66        
Total params: 2,786
Trainable params: 2,786
Non-trainable params: 0
_________________________________________________________________


In [6]:
# check toa-refl directory available

In [7]:
# get a list of toa_refl files

toa_refl_filePattern = "toa_refl_P*.dat"

toa_refl_list = glob(os.path.join(toa_refl_dir, toa_refl_filePattern))
print(len(toa_refl_list))


9


In [8]:
toa_refl_list

['/Users/ehsanmos/MLP_dataset/cloudmask_lab/prediction_test_cloudmask/test_cloudmask_p167_b10_july/toa_refl_P167_O088219_B010_da_red.dat',
 '/Users/ehsanmos/MLP_dataset/cloudmask_lab/prediction_test_cloudmask/test_cloudmask_p167_b10_july/toa_refl_P167_O088219_B010_ba_red.dat',
 '/Users/ehsanmos/MLP_dataset/cloudmask_lab/prediction_test_cloudmask/test_cloudmask_p167_b10_july/toa_refl_P167_O088219_B010_ca_red.dat',
 '/Users/ehsanmos/MLP_dataset/cloudmask_lab/prediction_test_cloudmask/test_cloudmask_p167_b10_july/toa_refl_P167_O088219_B010_an_red.dat',
 '/Users/ehsanmos/MLP_dataset/cloudmask_lab/prediction_test_cloudmask/test_cloudmask_p167_b10_july/toa_refl_P167_O088219_B010_af_red.dat',
 '/Users/ehsanmos/MLP_dataset/cloudmask_lab/prediction_test_cloudmask/test_cloudmask_p167_b10_july/toa_refl_P167_O088219_B010_df_red.dat',
 '/Users/ehsanmos/MLP_dataset/cloudmask_lab/prediction_test_cloudmask/test_cloudmask_p167_b10_july/toa_refl_P167_O088219_B010_bf_red.dat',
 '/Users/ehsanmos/MLP_datas

In [9]:
# make a list of POB form all available toa-refls to keep count of the number fo toa-refl files available

p_o_b_process_list = []

for toa_refl in toa_refl_list:
    #print(toa_refl)
    p_o_b_list = toa_refl.split('/')[-1].split('_')[2:5]
#     print(p_o_b_list)
    #print(file_name.split('_')[2:4])
#     p_o_list = file_name.split('_')[2:4]
    p_o_b = p_o_b_list[0]+"_"+p_o_b_list[1]+"_"+p_o_b_list[2]
#     print(p_o_b)

    if p_o_b in p_o_b_process_list:
        continue
    else:
        p_o_b_process_list.append(p_o_b)
        print("p-o-b was added to list")
        
        
p_o_b_process_list

p-o-b was added to list


['P167_O088219_B010']

In [10]:
## define and open 9 cameras in order
## based on order of input to MLP- based on order of cameras in training dataset
# later select toa-refls based on POB from inventory; then in the for-loop

# for-loop here for every POB in the p_o_b_process_list, open each 9 files together and process them

# for POB in p_o_b_process_list"
POB = p_o_b_process_list[0] # get the first one on the list manually


# cf_r = "toa_refl_"+p_o+"_B0"+toa_block+"_cf_red.dat"
da_r = 'toa_refl_'+POB+'_da_red.dat'
# print(da_r)
df_r = 'toa_refl_'+POB+'_df_red.dat'
ca_r = 'toa_refl_'+POB+'_ca_red.dat'
cf_r = 'toa_refl_'+POB+'_cf_red.dat'
ba_r = 'toa_refl_'+POB+'_ba_red.dat'
bf_r = 'toa_refl_'+POB+'_bf_red.dat'
aa_r = 'toa_refl_'+POB+'_aa_red.dat'
af_r = 'toa_refl_'+POB+'_af_red.dat'
an_r = 'toa_refl_'+POB+'_an_red.dat'



########### open and read toa-refl as np-arr
if (os.path.isfile(os.path.join(toa_refl_dir, an_r))==False):
    print("block not found- continue")
#     continue
an_r_arr = np.fromfile(os.path.join(toa_refl_dir, an_r), dtype=np.double)[0:1048576].reshape((512,2048))
## check black- continue:
if (np.median(an_r_arr) == -1.0):
    print("image black- continue")
#     continue



########### open and read toa-refl as np-arr
if (os.path.isfile(os.path.join(toa_refl_dir, aa_r))==False):
    print("block not found- continue")
#     continue
aa_r_arr = np.fromfile(os.path.join(toa_refl_dir, aa_r), dtype=np.double)[0:1048576].reshape((512,2048))
## check black- continue:
if (np.median(aa_r_arr) == -1.0):
    print("image black- continue")
#     continue



########### open and read toa-refl as np-arr
if (os.path.isfile(os.path.join(toa_refl_dir, af_r))==False):
    print("block not found- continue")
#     continue
af_r_arr = np.fromfile(os.path.join(toa_refl_dir, af_r), dtype=np.double)[0:1048576].reshape((512,2048))
## check black- continue:
if (np.median(af_r_arr) == -1.0):
    print("image black- continue")
#     continue



########### open and read toa-refl as np-arr
if (os.path.isfile(os.path.join(toa_refl_dir, ba_r))==False):
    print("block not found- continue")
#     continue
ba_r_arr = np.fromfile(os.path.join(toa_refl_dir, ba_r), dtype=np.double)[0:1048576].reshape((512,2048))
## check black- continue:
if (np.median(ba_r_arr) == -1.0):
    print("image black- continue")
#     continue


########### open and read toa-refl as np-arr
if (os.path.isfile(os.path.join(toa_refl_dir, bf_r))==False):
    print("block not found- continue")
#     continue
bf_r_arr = np.fromfile(os.path.join(toa_refl_dir, bf_r), dtype=np.double)[0:1048576].reshape((512,2048))
## check black- continue:
if (np.median(bf_r_arr) == -1.0):
    print("image black- continue")
#     continue



########### open and read toa-refl as np-arr
if (os.path.isfile(os.path.join(toa_refl_dir, ca_r))==False):
    print("block not found- continue")
#     continue
ca_r_arr = np.fromfile(os.path.join(toa_refl_dir, ca_r), dtype=np.double)[0:1048576].reshape((512,2048))
## check black- continue:
if (np.median(ca_r_arr) == -1.0):
    print("image black- continue")
#     continue



########### open and read toa-refl as np-arr
if (os.path.isfile(os.path.join(toa_refl_dir, cf_r))==False):
    print("block not found- continue")
#     continue
cf_r_arr = np.fromfile(os.path.join(toa_refl_dir, cf_r), dtype=np.double)[0:1048576].reshape((512,2048))
## check black- continue:
if (np.median(cf_r_arr) == -1.0):
    print("image black- continue")
#     continue


########### open and read toa-refl as np-arr
if (os.path.isfile(os.path.join(toa_refl_dir, da_r))==False):
    print("block not found- continue")
#     continue
da_r_arr = np.fromfile(os.path.join(toa_refl_dir, da_r), dtype=np.double)[0:1048576].reshape((512,2048))
## check black- continue:
if (np.median(da_r_arr) == -1.0):
    print("image black- continue")
#     continue


########### open and read toa-refl as np-arr
if (os.path.isfile(os.path.join(toa_refl_dir, df_r))==False):
    print("block not found- continue")
#     continue
df_r_arr = np.fromfile(os.path.join(toa_refl_dir, df_r), dtype=np.double)[0:1048576].reshape((512,2048))
## check black- continue:
if (np.median(df_r_arr) == -1.0):
    print("image black- continue")
#     continue


# we do not trip blocks, but if there is a bad value/black pixel in the input aarray, we fill that pixel with fillvalue==? from misr cloudmask filvalue



# prepare the input array to our model?



In [11]:
# check shapes of all arrays to make sure they all have same shape
an_r_arr.shape


(512, 2048)

In [12]:
# irow = 0
# icol = 100  
# path = 167
# orbit = 88219
# block = 10
# resolution = 275


# an_latlon = bls_to_latlon(path, resolution, block, 100, 1000)
# an_latlon

In [26]:
# # pixel matching ==> does not work; finding pixels based on similar latlons is very hard and not possible based on the matching latlons

# # open AN - ok
# # find the LS of the first pixel in the image
# resolution = 275
# an_latlon_list = []

# # for irow in range(0, an_r_arr.shape[0]):
# #     for icol in range(0, an_r_arr.shape[1]):
# #         an = an_r_arr[irow,icol]
# #         if (an >= 0): # vaues in array image are TOA reflectance
# #             # coolect index info as LS
# #             linsam = [irow,icol]
# # #             print(linsam)
# #             # change LS to location/latlon
# # #             bls_to_latlon(path, resolution meters, block, line, sample)
    
# #             latlon = bls_to_latlon(path, resolution, block, linsam[0], linsam[1])

# #             # find this latlon in other 8 cameras
# #             #ls_latlon = []
# #             linsam_latlon_an.append(linsam, latlon, an)

# irow = 100
# icol = 800  
# path = 167
# orbit = 88219
# block = 10
# linsam = [irow, icol]

# an = an_r_arr[irow, icol]
# # print(an)
# if (an >= 0): # vaues in array image are TOA reflectance
#     # coolect index info as LS
# #     linsam = [irow,icol]
# #             print(linsam)
#     # change LS to location/latlon

#     an_latlon = bls_to_latlon(path, resolution, block, linsam[0], linsam[1])
# #     print(an_latlon)
#     # change tuple to int seperately
#     an_lat = round(an_latlon[0], 4)
#     an_lon = round(an_latlon[1], 4)
#     print(an_lat)
#     # find this latlon in other 8 cameras
# #     an_latlon_list.append(list(an_latlon + (an,))) # how to add 2 tuples to a list
#     an_latlon_list.append([an_lat, an_lon, round(an, 6)]) # how to add 2 tuples to a list

    
# an_latlon_list         
# #####################   
# # da_latlon_list = []
# # for ii in range(512):
# #     for jj in range(2048):
# #         da = da_r_arr[ii,jj]
# #         if (da>=0):
# #             # also we need latlon for that pixel
# #             da_latlon = bls_to_latlon(path, resolution, block, ii, jj)
# #             da_latlon_list.append(da_latlon, da)
            
            
#             # 

# #####################   
# # now match an-latlon with da-latlon and find the da


        




# #####################               
# # # do this for all other 8 cameras
# # # find LS from latlons
# # for i in ls_latlon_an:
# #     x1 = ls_latlon_an[4] # ls_latlon_an should be a list w/5 elements: [l,s,lat,lon,an]
# #     an_latlon = ls_latlon_an[2:4]
#     # find this latlon in ls_latlon_aa and its corresponding aa and assign it to x2 
#     # find this latlon in ls_latlon_af and its corresponding af and assign it to x3 
#     # find this latlon in ls_latlon_bf and its corresponding bf and assign it to x4 

#     # later make this a lookup table?
            
            
            
            
            
#             # from latlon on AN, we find pixel values in other 8 cameras/arrays by matching latlon->LS:
#             # latlon -> different LS in each block -> find pixel value + add to lsit 
#             # convert LS to latlon for each camera
#             # match latlon AN with latlon in 8 other cameras and find corresponding LS pixel value for that latlon in 8 cameras
            
            
            
#             # use latlon to find LS in other images
            
#             # collect values of LS in other 8 cameras in a tuple
#             # add tuple to an image-list
    
    
# # LS to location information
# # use the location info to find pixel values of other 8 cameras
# # add them all into a tuple and in order of inputs to NN model, because they are input to our algorithm
# # after calculating this for a block, we do prediction for the block.






In [27]:
# da_latlon_list = []

# for ii in range(512):
#     for jj in range(2048):
#         da = da_r_arr[ii,jj]
#         if (da>=0):
#             # also we need latlon for that pixel
#             da_latlon = bls_to_latlon(path, resolution, block, ii, jj)
#             da_lat = round(da_latlon[0], 4)
#             da_lon = round(da_latlon[1], 4)
# #             da_latlon_list.append(list(da_latlon + (da,)))
#             da_latlon_list.append([da_lat, da_lon, round(da,6)])


In [19]:
# da_latlon_list
# an_latlon_list

In [22]:
# # now match an-latlon with da-latlon and find the da
# for irow in da_latlon_list:
# #     print(irow[0])
#     if (irow[0]== an_latlon_list[0] and irow[1]== an_latlon_list[1]):
#         da = irow[2]
#         print('pixel found')
#         print(da)


In [23]:
# # align 9 images in a way that every pixel sits on each other
# # how to remove the black sides of the imahes???????


# type(da_r_arr)

# # Create a mask to identify the desired values/region
# mask = (da_r_arr >= 0)
# # mask.shape

# # Use boolean indexing to get the non -1 values
# new_image = da_r_arr[mask]
# new_image.shape

# # # Reshape the 1D array back to a 2D array
# # num_rows = 512
# # num_cols = new_image.size // num_rows
# # # num_cols
# # new_image = new_image.reshape(num_rows, num_cols)

In [24]:
# visualize and check the cutted image
# histogram and check available values

ret_hist = plt.hist(new_image, bins='auto')
plt.show()


NameError: name 'new_image' is not defined

In [25]:
# # loop and extract every pixel from all 9 cameras for each row & column, 
# # it should be based on the order of inputs/cameras to the model(important)

# x_predict_block = []
# fillvalue_index = []

# for row in range(da_r_arr.shape[0]):
#     for col in range(da_r_arr.shape[1]):

#         # order of Xi should be based on the input orders to the our model
#         # order: feature_columns = ['anr','aa','af','ba','bf','ca','cf','da','df']
#         x1 = an_r_arr[row,col]
#         x2 = aa_r_arr[row,col]
#         x3 = af_r_arr[row,col]
#         x4 = ba_r_arr[row,col]
#         x5 = bf_r_arr[row,col]
#         x6 = ca_r_arr[row,col]
#         x7 = cf_r_arr[row,col]
#         x8 = da_r_arr[row,col]
#         x9 = df_r_arr[row,col]

#         # check somewhere here if a pixel was == -1, then replace all pixels with fillvalue and continue to the next pixel

        
#         nine_features = [x1,x2,x3,x4,x5,x6,x7,x8,x9]
#         # is -1 a good fillvalue for the edges?
#         # if -1 in the list:
#         # find the index, save the pixel index, and later replace w/fillvalue
#         if -1.0 in nine_features:
# #             print('-1 found in 9 features, save the index of this pixel.')
#             fillvalue_index.append((row*2048)+col) # similar to C code for counting index numbers

        
#         #print(type(nine_features))
#         x_predict_block.append(nine_features)
        

In [ ]:
# # apply the model and predict for a single block
# ## prediction for each block

# y_predict_block = mlp_model_best.predict(x_predict_block, verbose=1)

In [ ]:
# # change class categories to one class

# print(y_predict_block[100]) 

# # Set the threshold value to confirm classes
# threshold = 0.5

# # convert 1-hot prob to 1-hot encoding == binary classes
# cloud_mask_one_hot = np.zeros_like(y_predict_block)
# cloud_mask_one_hot[y_predict_block >= threshold] = 1
# print(cloud_mask_one_hot)

# # Convert 1-hot encoding to categorical 1-D output
# cloudmask_categorical = np.zeros(cloud_mask_one_hot.shape[0]) # make a zero column
# cloudmask_categorical[cloud_mask_one_hot[:, 1] == 1] = 1 # update the zero column==every pixel where it was originally 1
# print(cloudmask_categorical)

# # this is in 1-hot format? how to change it to normal format?


In [ ]:
# len(cloudmask_categorical)

In [ ]:
# # update the the predicted categorical list with fillvalue using indexing and assignment
# for i in fillvalue_index:
# #     print(i)
#     cloudmask_categorical[i] = -2 # -99 is fillvalue for a pixel in cloudmask that any camera was -1 in toa-refl==edges of image 


In [ ]:
# cloudmask_categorical

In [ ]:
# # change data type from float64 to int8
# cloudmask_categorical = cloudmask_categorical.astype(np.int8)
# cloudmask_categorical.dtype

In [ ]:
# ## re-construct cloudmask prediction list to 2D
# print(cloudmask_categorical.shape) # shape is 2 because we have 2 classes
# print(type(cloudmask_categorical))

# cloudmask_categorical_2d = cloudmask_categorical.reshape((512,-1))
# print(cloudmask_categorical_2d.dtype)
# print('output cloumask shape: (%s, %s)' %(cloudmask_categorical_2d.shape))

# print("min: %s" %cloudmask_categorical_2d.min())
# print("max: %s" %cloudmask_categorical_2d.max())

In [ ]:
# # histogram of data in cloudmask

# cloudmask_categorical_flat = cloudmask_categorical_2d.flatten()
# print(cloudmask_categorical_flat.shape)

# ret_hist = plt.hist(cloudmask_categorical_flat, bins='auto')
# plt.show()

In [ ]:
# maybe visualize cloudmask 

In [ ]:
# # p_o = 'P167_O088219'
# # toa_block = '10'

# out_raw_binary_label = 'cloudmask_'+POB+".msk"  # this image format supports saving neg- values in image
# out_raw_binary_fullpath = os.path.join(output_dir, out_raw_binary_label)


In [ ]:
# ## flatten array and save as binary raw file on local computer

# cloudmask_categorical_2d.flatten().astype(np.int8).tofile(out_raw_binary_fullpath) # write as uint8 to be able to read by atmmodel.c
# print(out_raw_binary_fullpath)
